In [1]:
!pip install faker
!pip install tqdm
!pip install keras_preprocessing
!pip install keras

     |████████████████████████████████| 1.0MB 4.9MB/s 


In [2]:
import sys
import requests
import tarfile
import json
import numpy as np
from os import path
from PIL import Image
from PIL import ImageFont, ImageDraw
from glob import glob
from matplotlib import pyplot as plt
import cv2

import random
from tqdm import tqdm

from faker import Faker
import re
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import CSVLogger

from keras.applications import MobileNetV2

from keras.layers import *
from keras.models import Model

Using TensorFlow backend.


In [3]:
#Variablen
img_w = 512
img_h = 512
channels = 3
classes = 1
info = 5
grid_w = 16
grid_h = 16

# Funktionen

In [4]:
# Function to viz the annotation

BildGrößeX = 512
BildGrößeY = 512

def markup(BildWeg, annotations):
  # laden der Zusatdaten
  Bild = cv2.imread(BildWeg)
  print(Bild)
  x_sl = BildGrößeX/Bild.shape[1]
  y_sl = BildGrößeY/Bild.shape[0]

  Bild = cv2.resize(Bild, (BildGrößeX, BildGrößeY))

  # laden der Infos
  Y = np.zeros((grid_h,grid_w,1,5))

  for annotation in annotations:
      # "bbox": [37.59, 433.64, 251.07, 20.38]
    xmin = annotation['bbox'][0] * x_sl
    xmax = (annotation['bbox'][0] + annotation['bbox'][2]) * x_sl

    ymin = annotation['bbox'][1] * y_sl
    ymax = (annotation['bbox'][1] + annotation['bbox'][3]) * y_sl
    
    w = (xmax - xmin)/BildGrößeX
    h = (ymax - ymin)/BildGrößeY
    
    x = ((xmax + xmin)/2)/BildGrößeX
    y = ((ymax + ymin)/2)/BildGrößeY
    x = x * grid_w
    y = y * grid_h

    Y[int(y),int(x),0,0] = 1
    Y[int(y),int(x),0,1] = x - int(x)
    Y[int(y),int(x),0,2] = y - int(y)
    Y[int(y),int(x),0,3] = w
    Y[int(y),int(x),0,4] = h

  return Bild, Y

# Model Funktionen


In [5]:
def yolo_model(input_shape):
    
    inp = Input(input_shape)
   
    model = MobileNetV2( input_tensor= inp , include_top=False, weights='imagenet')
    last_layer = model.output
    
    conv = Conv2D(512,(3,3) , activation='relu' , padding='same', name="ErstesCNNLayer")(last_layer)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    
    conv = Conv2D(128,(3,3) , activation='relu' , padding='same', name="ZweitesCNNLayer")(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    
    conv = Conv2D(5,(3,3) , activation='relu' , padding='same')(lr)
    
    final = Reshape((grid_h,grid_w,classes,info), name="Finale")(conv)
    
    model = Model(inp,final)
    
    return model

In [6]:
def yolo_loss_func(y_true,y_pred):
    # y_true : 16,16,1,5
    # y_pred : 16,16,1,5
    l_coords = 5.0
    l_noob = 0.5
    coords = y_true[:,:,:,:,0] * l_coords
    noobs = (-1*(y_true[:,:,:,:,0] - 1)*l_noob)
    p_pred = y_pred[:,:,:,:,0]
    p_true = y_true[:,:,:,:,0]
    x_true = y_true[:,:,:,:,1]
    x_pred = y_pred[:,:,:,:,1]
    yy_true = y_true[:,:,:,:,2]
    yy_pred = y_pred[:,:,:,:,2]
    w_true = y_true[:,:,:,:,3]
    w_pred = y_pred[:,:,:,:,3]
    h_true = y_true[:,:,:,:,4]
    h_pred = y_pred[:,:,:,:,4]
    
    p_loss_absent = K.sum(K.square(p_pred - p_true)*noobs)
    p_loss_present = K.sum(K.square(p_pred - p_true))
    x_loss = K.sum(K.square(x_pred - x_true)*coords)
    yy_loss = K.sum(K.square(yy_pred - yy_true)*coords)
    xy_loss = x_loss + yy_loss
    w_loss = K.sum(K.square(K.sqrt(w_pred) - K.sqrt(w_true))*coords)
    h_loss = K.sum(K.square(K.sqrt(h_pred) - K.sqrt(h_true))*coords)
    wh_loss = w_loss + h_loss
    
    loss = p_loss_absent + p_loss_present + xy_loss + wh_loss
    
    return loss

# Model Bau

In [7]:
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

#optimizer
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

csv_logger = CSVLogger('log.csv', append=True, separator=';')

#checkpoint
checkpoint = ModelCheckpoint('model1/text_detect.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min',period=1)

In [8]:
model = yolo_model((512,512,3))

model.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 1s 0us/step


# Training


In [9]:
def openTar(Name):
  tar = tarfile.open(Name)
  tar.extractall()
  tar.close()
  print(Name + " entpacked")


In [10]:
x = []
y = []

In [12]:
# Daten Laden
!curl https://dax-cdn.cdn.appdomain.cloud/dax-publaynet/1.0.0/train-0.tar.gz --output "train-0.tar.gz"
openTar("train-0.tar.gz")
!rm train-0.tar.gz

!curl https://dax-cdn.cdn.appdomain.cloud/dax-publaynet/1.0.0/train-1.tar.gz --output "train-1.tar.gz"
openTar("train-1.tar.gz")
!rm train-1.tar.gz

!curl https://dax-cdn.cdn.appdomain.cloud/dax-publaynet/1.0.0/labels.tar.gz --output "labels.tar.gz"
openTar("labels.tar.gz")
!rm labels.tar.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 30 12.6G   30 3922M    0     0  53.4M      0  0:04:03  0:01:13  0:02:50 43.6M^C


KeyboardInterrupt: ignored

In [ ]:
StringToJson = 'publaynet/train.json'
StringToFolder = 'publaynet/train/'

NumberPerTirm = 1000

# 335,703 Bilder

In [ ]:
testArray = []

with open(StringToJson, 'r') as fp:
    samples = json.load(fp)
# Index images
images = {}
for image in samples['images']:
    testArray.append(image['id'])
    images[image['id']] = {'file_name': StringToFolder + image['file_name'], 'annotations': []}
for ann in samples['annotations']:
    images[ann['image_id']]['annotations'].append(ann)

In [ ]:
def load(test):
  Train = []
  Lösung = []
  for i, (_, image) in enumerate(test.items()):
    try:
      TempImage, ZielMuster = markup(image['file_name'], image['annotations'])
      Train.append(TempImage)
      Lösung.append(ZielMuster)
    except Exception:
      continue

  Train = np.array(Train)
  Lösung = np.array(Lösung)
  return Train, Lösung

In [ ]:
for p in range(0,335703,NumberPerTirm):
  print("Starte bei Bild --------------- " + str(p))

  test = {}
  for _ in range(NumberPerTirm):
    id = testArray.pop(0)
    test[id] = images[id]

  x,y = load(test)

  print(len(x))

  model.fit(Train, Lösung, epochs= 32 ,batch_size = 4, validation_split=0.3 , callbacks=[csv_logger, checkpoint])

  model.save('model' + str(i))

# Auswertung


In [ ]:
plt.xkcd()

In [ ]:
Model_epoch = []
Model_loss = []
Model_val_loss = []

with open('log.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    header = next(spamreader)
    for row in spamreader:
         Wert = row[0]
         Model_epoch.append(float(Wert))
         Wert = row[1]
         Model_loss.append(float(Wert))
         Wert = row[2]
         Model_val_loss.append(float(Wert))

Model1_epoch = Model1_epoch[1:]
Model1_loss = Model1_loss[1:]
Model1_val_loss = Model1_val_loss[1:]

In [ ]:
plt.plot(Model_loss, label = 'val_loss')
plt.plot(Model_val_loss, label = 'loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 5])
plt.legend(loc='center right')
plt.show()

# Analyse

In [ ]:
# Daten Laden
!curl https://dax-cdn.cdn.appdomain.cloud/dax-publaynet/1.0.0/examples.tar.gz --output "examples.tar.gz"

openTar("examples.tar.gz")

StringToJSON = 'examples/samples.json'
StringToFolder = 'examples/'

x,y = loadData(StringToJSON, StringToFolder)